# Introduction

## Problem:
## Garbage In => Garbage out

Anecdote story:
Demand prediction system for online retailer.


One data source provided information about promotion campaigns, e.g.

How it was:
```
"" -> No campaign
"Sping_2018" -> sku included in promotion campaign "Spring 2018"
```

Generated boolean feature from it:
``bool(len(column_name))``

Then they silently changed it to :
```
"None" -> No campaign
"Sping_2018" -> sku included in promotion campaign "Spring 2018"
```

In [66]:
import pandera as pa
from pandera import Column, Check
import numpy as np

In [72]:
import pandas as pd

fruits = pd.DataFrame(
    {
        "name": ["apple", "banana", "apple", "orange"],
        "store": ["Aldi", "Walmart", "Walmart", "Aldi"],
        "price": [2, 1, 3, 4],
    }
)

fruits

,name,store,price
0,apple,Aldi,2
1,banana,Walmart,1
2,apple,Walmart,3
3,orange,Aldi,4


## [Pandera website](https://union.ai/pandera)

# DataFrame Schema

In [81]:
available_fruits = ["apple", "banana", "orange"]
nearby_stores = ["Aldi", "Walmart"]

schema = pa.DataFrameSchema(
    {
        "name": Column(str, Check.isin(available_fruits)),
        "store": Column(str, Check.isin(nearby_stores)),
        "price": Column(int, Check.less_than(3)),
    }
)

In [82]:
schema.validate(fruits)

SchemaError: <Schema Column(name=price, type=DataType(int64))> failed element-wise validator 0:
<Check less_than: less_than(3)>
failure cases:
   index  failure_case
0      2             3
1      3             4

In [83]:
schema = pa.DataFrameSchema(
    {
        "name": Column(str, Check.isin(available_fruits)),
        "store": Column(str, Check.isin(nearby_stores)),
        "price": Column(
            int,
            [
                Check.less_than(5),
                Check(lambda price: np.mean(price) < 4, element_wise=False),
            ],
        ),
    }
)
schema.validate(fruits)


,name,store,price
0,apple,Aldi,2
1,banana,Walmart,1
2,apple,Walmart,3
3,orange,Aldi,4


### More checks in the [documentation](https://pandera.readthedocs.io/en/stable/checks.html)

# Schema Model

In [71]:
from pandera.typing import Series


class Schema(pa.SchemaModel):
    name: Series[str] = pa.Field(isin=available_fruits)
    store: Series[str] = pa.Field(isin=nearby_stores)
    price: Series[int] = pa.Field(le=5)

    @pa.check("price")
    def price_sum_lt_20(cls, price: Series[int]) -> Series[bool]:
        return np.sum(price) < 20

    @pa.check("price")
    def price_mean_lt_5(cls, price: Series[int]) -> Series[bool]:
        return np.mean(price) < 5

Schema.validate(fruits)

,name,store,price
0,apple,Aldi,2
1,banana,Walmart,1
2,apple,Walmart,3
3,orange,Aldi,4


# Validation Decorator 

## Check Input

In [43]:
fruits = pd.DataFrame(
    {
        "name": ["apple", "banana", "apple", "orange"],
        "store": ["Aldi", "Walmart", "Walmart", "Aldi"],
        "price": [2, 1, 3, 4],
    }
)

schema = pa.DataFrameSchema(
    {
        "name": Column(str, Check.isin(available_fruits)),
        "store": Column(str, Check.isin(nearby_stores)),
        "price": Column(int, Check.less_than(5)),
    }
)


def get_total_price(fruits: pd.DataFrame, schema: pa.DataFrameSchema):
    validated = schema.validate(fruits)
    return validated["price"].sum()


get_total_price(fruits, schema)

10

In [44]:
from pandera import check_input, check_output, check_io

@check_input(schema)
def get_total_price(fruits: pd.DataFrame):
    return fruits.price.sum()


get_total_price(fruits)

10

## Check Output

In [30]:
fruits_nearby = pd.DataFrame(
    {
        "name": ["apple", "banana", "apple", "orange"],
        "store": ["Aldi", "Walmart", "Walmart", "Aldi"],
        "price": [2, 1, 3, 4],
    }
)

fruits_faraway = pd.DataFrame(
    {
        "name": ["apple", "banana", "apple", "orange"],
        "store": ["Whole Foods", "Whole Foods", "Schnucks", "Schnucks"],
        "price": [3, 2, 4, 5],
    }
)

out_schema = pa.DataFrameSchema(
    {"store": Column(str, Check.isin(["Aldi", "Walmart", "Whole Foods", "Schnucks"]))}
)


@check_output(out_schema)
def combine_fruits(fruits_nearby: pd.DataFrame, fruits_faraway: pd.DataFrame):
    fruits = pd.concat([fruits_nearby, fruits_faraway])
    return fruits


combine_fruits(fruits_nearby, fruits_faraway)

,name,store,price
0,apple,Aldi,2
1,banana,Walmart,1
2,apple,Walmart,3
3,orange,Aldi,4
0,apple,Whole Foods,3
1,banana,Whole Foods,2
2,apple,Schnucks,4
3,orange,Schnucks,5


<IPython.core.display.Javascript object>

## Check Both

In [31]:
in_schema = pa.DataFrameSchema({"store": Column(str)})

out_schema = pa.DataFrameSchema(
    {"store": Column(str, Check.isin(["Aldi", "Walmart", "Whole Foods", "Schnucks"]))}
)


@check_io(fruits_nearby=in_schema, fruits_faraway=in_schema, out=out_schema)
def combine_fruits(fruits_nearby: pd.DataFrame, fruits_faraway: pd.DataFrame):
    fruits = pd.concat([fruits_nearby, fruits_faraway])
    return fruits


combine_fruits(fruits_nearby, fruits_faraway)

,name,store,price
0,apple,Aldi,2
1,banana,Walmart,1
2,apple,Walmart,3
3,orange,Aldi,4
0,apple,Whole Foods,3
1,banana,Whole Foods,2
2,apple,Schnucks,4
3,orange,Schnucks,5


<IPython.core.display.Javascript object>

# Other Arguments for Column Validation

## Deal with Null Values

In [46]:
fruits = pd.DataFrame(
    {
        "name": ["apple", "banana", "apple", "orange"],
        "store": ["Aldi", "Walmart", "Walmart", np.nan],
        "price": [2, 1, 3, 4],
    }
)

fruits

,name,store,price
0,apple,Aldi,2
1,banana,Walmart,1
2,apple,Walmart,3
3,orange,NaN,4


In [47]:
schema = pa.DataFrameSchema(
    {
        "name": Column(str, Check.isin(available_fruits)),
        "store": Column(
            str,
            Check.isin(nearby_stores),
            nullable=True,
        ),
        "price": Column(int, Check.less_than(5)),
    }
)
schema.validate(fruits)


,name,store,price
0,apple,Aldi,2
1,banana,Walmart,1
2,apple,Walmart,3
3,orange,NaN,4


# Deal with data types

In [49]:
fruits = pd.DataFrame(
    {
        "name": ["apple", "banana", "apple", "orange"],
        "store": ["Aldi", "Walmart", "Walmart", "Aldi"],
        "price": ["2", "1", "3", "4"],
    }
)

In [50]:
schema = pa.DataFrameSchema(
    {
        "name": Column(str, Check.isin(available_fruits)),
        "store": Column( str, Check.isin(nearby_stores)),
        "price": Column(
            int,
            Check.less_than(5),
            coerce=True
        ),
    }
)
schema.validate(fruits)


,name,store,price
0,apple,Aldi,2
1,banana,Walmart,1
2,apple,Walmart,3
3,orange,Aldi,4


## Deal with Duplicates

In [53]:
schema = pa.DataFrameSchema(
    {
        "name": Column(str, Check.isin(available_fruits)),
        "store": Column(
            str,
            Check.isin(nearby_stores),
            nullable=True,
            unique=True
        ),
        "price": Column(int, Check.less_than(5)),
    }
)
schema.validate(fruits)


SchemaError: series 'store' contains duplicate values:
0       Aldi
1    Walmart
2    Walmart
3       Aldi
Name: store, dtype: object

## Convert Data Types

In [52]:
fruits = pd.DataFrame(
    {
        "name": ["apple", "banana", "apple", "orange"],
        "store": ["Aldi", "Walmart", "Walmart", "Aldi"],
        "price": [2, 1, 3, 4],
    }
)

schema = pa.DataFrameSchema({"price": Column(str, coerce=True)})
validated = schema.validate(fruits)
validated.dtypes

name     object
store    object
price    object
dtype: object

## Patern Matching

In [56]:
favorite_stores = ["Aldi", "Walmart", "Whole Foods", "Schnucks"]

fruits = pd.DataFrame(
    {
        "name": ["apple", "banana", "apple", "orange"],
        "store_nearby": ["Aldi", "Walmart", "Walmart", "Aldi"],
        "store_far": ["Whole Foods", "Schnucks", "Whole Foods", "Schnucks"],
    }
)

schema = pa.DataFrameSchema(
    {
        "name": Column(str, Check.isin(available_fruits)),
        "store_*": Column(
            str,
            Check.isin(favorite_stores),
            regex=True
        ),
    }
)
schema.validate(fruits)


,name,store_nearby,store_far
0,apple,Aldi,Whole Foods
1,banana,Walmart,Schnucks
2,apple,Walmart,Whole Foods
3,orange,Aldi,Schnucks


# Export and Load From a YAML file

## Export

In [58]:
yaml_schema = schema.to_yaml()
print(yaml_schema)

schema_type: dataframe
version: 0.16.1
columns:
  name:
    title: null
    description: null
    dtype: str
    nullable: false
    checks:
      isin:
      - apple
      - banana
      - orange
    unique: false
    coerce: false
    required: true
    regex: false
  store_*:
    title: null
    description: null
    dtype: str
    nullable: false
    checks:
      isin:
      - Aldi
      - Walmart
      - Whole Foods
      - Schnucks
    unique: false
    coerce: false
    required: true
    regex: true
checks: null
index: null
dtype: null
coerce: false
strict: false
name: null
ordered: false
unique: null
report_duplicates: all
unique_column_names: false
add_missing_columns: false
title: null
description: null



In [61]:
from pathlib import Path

f = Path("schema.yml")
f.touch()
f.write_text(yaml_schema)

725

## Load

In [62]:
with f.open() as file:
    yaml_schema = file.read()

In [63]:
schema = pa.io.from_yaml(yaml_schema)
schema

<Schema DataFrameSchema(columns={'name': <Schema Column(name=name, type=DataType(str))>, 'store_*': <Schema Column(name=store_*, type=DataType(str))>}, checks=[], index=None, coerce=False, dtype=None, strict=False, name=None, ordered=False, unique_column_names=Falsemetadata='None, unique_column_names=False, add_missing_columns=False)>

# [Original article](https://towardsdatascience.com/validate-your-pandas-dataframe-with-pandera-2995910e564)